In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

def clean(author):
    
    new = re.sub('\n|\t|\r', '', author)
    new = new = re.sub('\s{2,}', ' ', new)
    return new


def sibling_cleaner(sibling_list):
    
    i = 0
    while i < len(sibling_list):
        
        if sibling_list[i] == '\n':
            del sibling_list[i]
            i -= 1
            
        i += 1
    return sibling_list

In [ ]:
# neeche ye rha
dic = {'urls':[]}
driver = webdriver.Chrome()
driver.get('https://www.aasld.org/the-liver-meeting/schedule-and-speakers')


In [ ]:
page = 0
flag = True
while flag:
    print(f'page no . :- {page}=============================================\n')
    
    time.sleep(6)
    home_page = driver.page_source
    homesoup = BeautifulSoup(home_page, 'lxml')
    

    for hpg in homesoup.find_all(class_='info-grid__box'):
        if hpg.find(class_='info-grid__title').get('href'):
            session_link = 'https://www.aasld.org' + hpg.find(class_='info-grid__title').get('href')
            print(session_link)
            dic['urls'].append(session_link)

    # find next button
    
    for cl in driver.find_elements(By.TAG_NAME, 'button'):
        if cl.text == 'Next' and cl.is_enabled():
            found = True
            cl.click()
            time.sleep(7)
        elif cl.text == 'Next' and cl.is_enabled() == False:
            flag = False
    
    page += 1
    

In [ ]:
dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_id':[],
    'news_type':[],
    'session_title':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[]
    }

In [ ]:
c = 0
se_no = 1
for ou in df['urls']:
    
    session_id = f'S{se_no}'
    print(ou)
    req = requests.get(ou)
    soup = BeautifulSoup(req.content, 'lxml')

    session_title = soup.find(class_='col-12 col-lg-7').find('h3').text.strip() # session title
    presentations = []
    # session date time and location
    session_date = ''
    session_time = ''
    session_location = ''

    date_time_room = clean(soup.find(class_='col-12 col-lg-7').find('p').text)

    if re.search('Date.*', date_time_room, flags=re.S):
        session_date = re.search('Date:(.*,.*\d[A-Za-z]{1,2})', date_time_room, flags=re.S).group(1).strip()

    if re.search('Time:.*', date_time_room, flags=re.S):
        session_time = re.search('Time:(.\d{1,2}:\d{1,2}.*- \d{1,2}:\d{1,2} [A-Z]{1,2})',date_time_room, flags=re.S).group(1).strip()

    if re.search('Room:.*', date_time_room, flags=re.S):
        session_location = re.search('Room:(.*)', date_time_room, flags=re.S).group(1).strip()

    # abstract text
    session_text = ''
    for i in sibling_cleaner(list(soup.find(class_='col-12 col-lg-7').find('p').next_siblings)):

        ul_text = ''
        if i.name == 'ul':
            for li in i.find_all('li'):
                ul_text += li.text + ' '
            if ul_text:
                session_text += ul_text + ' '

        else:
            session_text += i.text + ' '
            
        #check for presentations...
        for pre in i.find_all('a'):
            if 'http' not in str(pre.get('href')):
                presentations.append(pre.get('href'))


    # session author
    session_author = ''
    session_aff = ''
    if soup.find(class_='cta-block__content'):
        for i in soup.find(class_='cta-block__content').find_all('a'):
            given = i.text.strip()
            to_find = str(i.get('href'))

            sten = to_find.split('-')[-1][0].upper() + to_find.split('-')[-1][1:-1] + to_find.split('-')[-1][-1].upper()
            if len(given.split(to_find.split('-')[-1].upper())) > 1:
                session_author += given.split(to_find.split('-')[-1].upper())[0].strip()[:-1] + ' '+ to_find.split('-')[-1].upper() + '; '
                session_aff += given.split(to_find.split('-')[-1].upper())[1].strip()[1:] + '; '



            elif len(given.split(to_find.split('-')[-1].capitalize())) > 1:

                session_author += given.split(to_find.split('-')[-1].capitalize())[0].strip()[:-1] + ' '+ to_find.split('-')[-1].capitalize() + '; '
                session_aff += given.split(to_find.split('-')[-1].capitalize())[1].strip()[1:] + '; '


            elif len(given.split(to_find.split('-')[-1])) > 1:

                session_author += given.split(to_find.split('-')[-1])[0].strip()[:-1] + ' ' + to_find.split('-')[-1] + '; '
                session_aff += given.split(to_find.split('-')[-1])[1].strip()[1:] + '; '

            elif len(given.split(sten)) > 1:
                session_author += given.split(sten)[0].strip()[:-1] + ' ' + sten + '; '
                session_aff += given.split(sten)[1].strip()[1:] + '; '

    if presentations:
        session_text = re.sub('Abstracts.*', '', session_text, flags=re.S)
        
    if len(session_time.split('-')[0].split()) == 1:
        session_time = session_time.split('-')[0].split()[0] + ' ' + session_time.split('-')[1].split()[1] + ' - ' + session_time.split('-')[1].split()[0] + ' ' + session_time.split('-')[1].split()[1]

    
    print(f'sess title :- {session_title}')
    print(f'sess date :- {session_date}')
    print(f'sess time :- {session_time}')
    print(f'sess loc :- {session_location}')
    print(f'sess author :- {session_author}')
    print(f'sess aff :- {session_aff}')
    print(f'sess text :- {session_text}')

    print('========='*13, c)

    dic['session_id'].append(session_id)
    dic['news_type'].append('Session')
    dic['source_id'].append('')
    dic['manual_id'].append('')
    dic['article_title'].append(session_title)
    dic['url'].append(ou)
    dic['authors'].append(session_author)
    dic['author_affiliation'].append(session_aff)
    dic['abstract_text'].append(session_text)
    dic['date'].append(session_date)
    dic['start_time'].append(session_time)
    dic['end_time'].append('')
    dic['location'].append(session_location)
    dic['session_title'].append(session_title)
    dic['session_type'].append('')
    dic['category'].append('')
    dic['sub_category'].append('')
    dic['disclosure'].append('')
    if presentations:
        print(session_link)
        try:
        
            session_title = df_data.iloc[c]['session_title']
            session_location = df_data.iloc[c]['location']
            session_type = df_data.iloc[c]['session_type']
            session_category = df_data.iloc[c]['category']
            sub_cat = df_data.iloc[c]['sub_category']
            session_date = df_data.iloc[c]['date']
            print(session_title)


            for p in presentations:
                abstract_link = f'https://www.aasld.org{p}'
                print(abstract_link)
                print(c)

                req = requests.get(abstract_link)
                soup = BeautifulSoup(req.content, 'lxml')
                article_title = soup.find(class_='col-12 col-lg-7').find('h3').text.strip() # session title

                article_time = ''
                date_time_room = clean(soup.find(class_='cta-block__content').text)

                if re.search('Time:.*', date_time_room, flags=re.S):
                    article_time = re.search('Time:(.\d{1,2}:\d{1,2}.*- \d{1,2}:\d{1,2} [A-Z]{1,2})',date_time_room, flags=re.S).group(1).strip()


                # abstract text
                article_text = ''
                new_text = [soup.find(class_='col-12 col-lg-7').find('p')] + list(soup.find(class_='col-12 col-lg-7').find('p').next_siblings)
                for i in sibling_cleaner(new_text):

                    ul_text = ''
                    if i.name == 'ul':
                        for li in i.find_all('li'):
                            ul_text += li.text + ' '
                        if ul_text:
                            article_text += ul_text + ' '

                    else:
                        article_text += i.text + ' '

                # session author
                article_author = ''
                article_aff = ''
                if soup.find(class_='cta-block__content'):
                    for i in soup.find(class_='cta-block__content').find_all('a')[:-1]:
                        given = i.text.strip()
                        to_find = str(i.get('href'))

                        sten = to_find.split('-')[-1][0].upper() + to_find.split('-')[-1][1:-1] + to_find.split('-')[-1][-1].upper()
                        if len(given.split(to_find.split('-')[-1].upper())) > 1:
                            article_author += given.split(to_find.split('-')[-1].upper())[0].strip()[:-1] + ' '+ to_find.split('-')[-1].upper() + '; '
                            article_aff += given.split(to_find.split('-')[-1].upper())[1].strip()[1:] + '; '


                        elif len(given.split(to_find.split('-')[-1].capitalize())) > 1:

                            article_author += given.split(to_find.split('-')[-1].capitalize())[0].strip()[:-1] + ' '+ to_find.split('-')[-1].capitalize() + '; '
                            article_aff += given.split(to_find.split('-')[-1].capitalize())[1].strip()[1:] + '; '


                        elif len(given.split(to_find.split('-')[-1])) > 1:

                            article_author += given.split(to_find.split('-')[-1])[0].strip()[:-1] + ' ' + to_find.split('-')[-1] + '; '
                            article_aff += given.split(to_find.split('-')[-1])[1].strip()[1:] + '; '

                        elif len(given.split(sten)) > 1:
                            article_author += given.split(sten)[0].strip()[:-1] + ' ' + sten + '; '
                            article_aff += given.split(sten)[1].strip()[1:] + '; '

                if len(article_time.split('-')[0].split()) == 1:
                    article_time = article_time.split('-')[0].split()[0] + ' ' + article_time.split('-')[1].split()[1] + ' - ' + article_time.split('-')[1].split()[0] + ' ' + article_time.split('-')[1].split()[1]

                print(f'sess title :- {article_title}')
                print(f'sess time :- {article_time}')
                print(f'sess author :- {article_author}')
                print(f'sess aff :- {article_aff}')
                print(f'sess text :- {article_text}')
                print('=8=8=8='*17)

                dic['session_id'].append(session_id)
                dic['news_type'].append('Abstract')
                dic['source_id'].append('')
                dic['manual_id'].append('')
                dic['article_title'].append(article_title)
                dic['url'].append(abstract_link)
                dic['authors'].append(article_author)
                dic['author_affiliation'].append(article_aff)
                dic['abstract_text'].append(article_text)
                dic['date'].append(session_date)
                dic['start_time'].append(article_time)
                dic['end_time'].append('')
                dic['location'].append(session_location)
                dic['session_title'].append(session_title)
                dic['session_type'].append('')
                dic['category'].append('')
                dic['sub_category'].append('')
                dic['disclosure'].append('')
                
                
        except:
            pass
    
    c += 1
    se_no += 1

In [ ]:
dfdata = pd.DataFrame(dic)
dfdata

In [ ]:
dfdata.to_excel('aasld_dat.xlsx', index=False)

In [ ]:
sessdic = dict()
sesstopic = dict()
sessprogtype = dict()

In [ ]:
# program type filter krke sare link upr ke dict me update 

flag = True
while flag:
    
    if 'Transplant/Liver Failure' in sesstopic:

        print('in if')
        time.sleep(5)
        home_page = driver.page_source
        homesoup = BeautifulSoup(home_page, 'lxml')

        for hpg in homesoup.find_all(class_='info-grid__box'):
            if hpg.find(class_='info-grid__title').get('href'):
                session_link = 'https://www.aasld.org' + hpg.find(class_='info-grid__title').get('href')
                print(session_link)
                sesstopic['Transplant/Liver Failure'].append(session_link)
                
        for cl in driver.find_elements(By.TAG_NAME, 'button'):
            if cl.text == 'Next' and cl.is_enabled():
                cl.click()
                time.sleep(6)
            elif cl.text == 'Next' and cl.is_enabled() == False:
                flag = False


    else:
        print('in else')
        home_page = driver.page_source
        homesoup = BeautifulSoup(home_page, 'lxml')

        links = []
        for hpg in homesoup.find_all(class_='info-grid__box'):
            if hpg.find(class_='info-grid__title').get('href'):
                session_link = 'https://www.aasld.org' + hpg.find(class_='info-grid__title').get('href')
                print(session_link)
                links.append(session_link)

        sesstopic.update({'Transplant/Liver Failure':links})
        
        found = False
        for cl in driver.find_elements(By.TAG_NAME, 'button'):
            if cl.text == 'Next' and cl.is_enabled():
                found = True
                cl.click()
                time.sleep(6)
            elif cl.text == 'Next' and cl.is_enabled() == False:
                flag = False
                
        flag = found

In [ ]:
sesstopic

In [ ]:
# dump it just in case we get problem with kernal
with open('aasld_json_sesstopic.json', 'w') as f:
    json.dump(sesstopic, f)

In [ ]:
urls_links = set(df['urls'])
# saare link already hai df urls me to abhi sabko sesstype or category do
for i in sesstopic:
    for j in sesstopic[i]:
        print(j in urls_links)

In [ ]:
# crawled data me category or sesstype bharo
df_data = pd.read_excel('aasld_dat.xlsx')
df_data

In [ ]:
# category filling
for t in sesstopic:
    for u in sesstopic[t]:
        # apna data
        c = 0
        for d in df_data['url']:
            if str(u) == str(d):
                try:
                    df_data.iat[c, df_data.columns.get_loc('category')] += f'; {t}'
                except:
                    df_data.iat[c, df_data.columns.get_loc('category')] = f'{t}'
            c += 1

In [ ]:
# sesstype filling
for t in sessdic:
    print(t)
    for u in sessdic[t]:
        # apna data
        c = 0
        for d in df_data['url']:
            if str(u) == str(d):
                try:
                    df_data.iat[c, df_data.columns.get_loc('session_type')] += f'; {t}'
                except:
                    df_data.iat[c, df_data.columns.get_loc('session_type')] = f'{t}'
            c += 1

In [ ]:
# pta nhi filling
for t in sessprogtype:
    print(t)
    for u in sessprogtype[t]:
        # apna data
        c = 0
        for d in df_data['url']:
            if str(u) == str(d):
                try:
                    df_data.iat[c, df_data.columns.get_loc('sub_category')] += f'; {t}'
                except:
                    df_data.iat[c, df_data.columns.get_loc('sub_category')] = f'{t}'
            c += 1

In [ ]:
# final excel with evryshit
df_data.to_excel('aasld_final.xlsx', index=False)

In [ ]:
ch_sess = []
dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_title':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[]
    }

In [ ]:
c = 0

for ou in df_data['url']:
    presentations = []
    req = requests.get(ou)
    soup = BeautifulSoup(req.content, 'lxml')
    for i in sibling_cleaner(list(soup.find(class_='col-12 col-lg-7').find('p').next_siblings)):
        for pre in i.find_all('a'):
            if 'http' not in str(pre.get('href')):
                presentations.append(pre.get('href'))

    if presentations:
        print(ou)
        try:
        
            session_title = df_data.iloc[c]['session_title']
            session_location = df_data.iloc[c]['location']
            session_type = df_data.iloc[c]['session_type']
            session_category = df_data.iloc[c]['category']
            sub_cat = df_data.iloc[c]['sub_category']
            session_date = df_data.iloc[c]['date']
            print(session_title)


            for p in presentations:
                abstract_link = f'https://www.aasld.org{p}'
                print(abstract_link)
                print(c)

                req = requests.get(abstract_link)
                soup = BeautifulSoup(req.content, 'lxml')
                article_title = soup.find(class_='col-12 col-lg-7').find('h3').text.strip() # session title

                article_time = ''
                date_time_room = clean(soup.find(class_='cta-block__content').text)

                if re.search('Time:.*', date_time_room, flags=re.S):
                    article_time = re.search('Time:(.\d{1,2}:\d{1,2}.*- \d{1,2}:\d{1,2} [A-Z]{1,2})',date_time_room, flags=re.S).group(1).strip()


                # abstract text
                article_text = ''
                new_text = [soup.find(class_='col-12 col-lg-7').find('p')] + list(soup.find(class_='col-12 col-lg-7').find('p').next_siblings)
                for i in sibling_cleaner(new_text):

                    ul_text = ''
                    if i.name == 'ul':
                        for li in i.find_all('li'):
                            ul_text += li.text + ' '
                        if ul_text:
                            article_text += ul_text + ' '

                    else:
                        article_text += i.text + ' '

                # session author
                article_author = ''
                article_aff = ''
                if soup.find(class_='cta-block__content'):
                    for i in soup.find(class_='cta-block__content').find_all('a')[:-1]:
                        given = i.text.strip()
                        to_find = str(i.get('href'))

                        sten = to_find.split('-')[-1][0].upper() + to_find.split('-')[-1][1:-1] + to_find.split('-')[-1][-1].upper()
                        if len(given.split(to_find.split('-')[-1].upper())) > 1:
                            article_author += given.split(to_find.split('-')[-1].upper())[0].strip()[:-1] + ' '+ to_find.split('-')[-1].upper() + '; '
                            article_aff += given.split(to_find.split('-')[-1].upper())[1].strip()[1:] + '; '


                        elif len(given.split(to_find.split('-')[-1].capitalize())) > 1:

                            article_author += given.split(to_find.split('-')[-1].capitalize())[0].strip()[:-1] + ' '+ to_find.split('-')[-1].capitalize() + '; '
                            article_aff += given.split(to_find.split('-')[-1].capitalize())[1].strip()[1:] + '; '


                        elif len(given.split(to_find.split('-')[-1])) > 1:

                            article_author += given.split(to_find.split('-')[-1])[0].strip()[:-1] + ' ' + to_find.split('-')[-1] + '; '
                            article_aff += given.split(to_find.split('-')[-1])[1].strip()[1:] + '; '

                        elif len(given.split(sten)) > 1:
                            article_author += given.split(sten)[0].strip()[:-1] + ' ' + sten + '; '
                            article_aff += given.split(sten)[1].strip()[1:] + '; '

                if len(article_time.split('-')[0].split()) == 1:
                    article_time = article_time.split('-')[0].split()[0] + ' ' + article_time.split('-')[1].split()[1] + ' - ' + article_time.split('-')[1].split()[0] + ' ' + article_time.split('-')[1].split()[1]

                print(f'sess title :- {article_title}')
                print(f'sess time :- {article_time}')
                print(f'sess author :- {article_author}')
                print(f'sess aff :- {article_aff}')
                print(f'sess text :- {article_text}')
                print('=8=8=8='*17)

                dic['source_id'].append('')
                dic['manual_id'].append('')
                dic['article_title'].append(article_title)
                dic['url'].append(abstract_link)
                dic['authors'].append(article_author)
                dic['author_affiliation'].append(article_aff)
                dic['abstract_text'].append(article_text)
                dic['date'].append(session_date)
                dic['start_time'].append(article_time)
                dic['end_time'].append('')
                dic['location'].append(session_location)
                dic['session_title'].append(session_title)
                dic['session_type'].append(session_type)
                dic['category'].append(session_category)
                dic['sub_category'].append(sub_cat)
                dic['disclosure'].append('')
                
                ch_sess.append(ou)
        except:
            pass
    c += 1

In [ ]:
df_p = pd.DataFrame(dic)
df_p

In [ ]:
df_data = pd.read_excel('aasld_final.xlsx')

In [ ]:
res = pd.concat([df_data, df_p])
res

In [ ]:
res.to_excel('aasld_withpress.xlsx', index=False)